# <font color=darkblue> Machine Learning model deployment with Flask framework</font>

## <font color=Blue>Used Cars Price Prediction Application</font>

### Objective:
1. To build a Machine learning regression model to predict the selling price of the used cars based on the different input features like fuel_type, kms_driven, type of transmission etc.
2. Deploy the machine learning model with the help of the flask framework.

### Dataset Information:
#### Dataset Source: https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=CAR+DETAILS+FROM+CAR+DEKHO.csv
This dataset contains information about used cars listed on www.cardekho.com
- **Car_Name**: Name of the car
- **Year**: Year of Purchase
- **Selling Price (target)**: Selling price of the car in lakhs
- **Present Price**: Present price of the car in lakhs
- **Kms_Driven**: kilometers driven
- **Fuel_Type**: Petrol/diesel/CNG
- **Seller_Type**: Dealer or Indiviual
- **Transmission**: Manual or Automatic
- **Owner**: first, second or third owner


### 1. Import required libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### 2. Load the dataset

In [2]:
df = pd.read_csv('car+data.csv')
df.head(5)

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


### 3. Check the shape and basic information of the dataset.

In [3]:
## check the shape
df.shape

(301, 9)

In [4]:
## check basic information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


- There are total 9 columns and 301 entries in each column
- of these, 2 are float datatype, 3 are int datatype and 4 are of object datatype

### 4. Check for the presence of the duplicate records in the dataset? If present drop them

In [5]:
# checking number of duplicate records
len(df[df.duplicated()])

2

- We can observe that there are 2 duplicate records

In [6]:
# Dropping the duplicate records
df = df.drop_duplicates()

In [7]:
# checking the length after dropping duplicate records
len(df[df.duplicated()])

0

### 5. Drop the columns which you think redundant for the analysis.

In [8]:
# dropping columns which are redundant for analysis
df.drop(['Car_Name'],axis=1, inplace=True)

In [9]:
df.head(2)

,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0


### 6. Extract a new feature called 'age_of_the_car' from the feature 'year' and drop the feature year

In [10]:
# subtracting the year of purchase with current year to get age of car
df['age_of_the_car'] = 2022-df['Year']

In [11]:
# dropping column year
df = df.drop('Year', axis=1)

In [12]:
df.head(2)

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,age_of_the_car
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,8
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,9


### 7. Encode the categorical columns

In [13]:
df['Fuel_Type'].unique()

array(['Petrol', 'Diesel', 'CNG'], dtype=object)

In [14]:
df['Seller_Type'].unique()

array(['Dealer', 'Individual'], dtype=object)

In [15]:
df['Transmission'].unique()

array(['Manual', 'Automatic'], dtype=object)

In [16]:
## Manual encoding:
df['Fuel_Type'] = df['Fuel_Type'].replace({'Petrol':0, 'Diesel':1,'CNG':2})
df['Seller_Type'] = df['Seller_Type'].replace({'Dealer':0, 'Individual':1})
df['Transmission'] = df['Transmission'].replace({'Manual':0, 'Automatic':1})

In [17]:
df['Fuel_Type'].unique()

array([0, 1, 2], dtype=int64)

In [18]:
df['Seller_Type'].unique()

array([0, 1], dtype=int64)

In [19]:
df['Transmission'].unique()

array([0, 1], dtype=int64)

### 8. Separate the target and independent features.

In [20]:
X = df.drop('Selling_Price', axis=1)
y = df['Selling_Price']

### 9. Split the data into train and test.

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(209, 7) (90, 7)
(209,) (90,)


### 10. Build a Random forest Regressor model and check the r2-score for train and test.

In [22]:
# Build a Random Forest Regressor model
rfr = RandomForestRegressor() 

# Fit the model to the training data
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [23]:
# Predict on the training and testing data
y_train_pred = rfr.predict(X_train)
y_test_pred = rfr.predict(X_test)

# Calculate the R-squared score for the training and testing data
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

# Print the R-squared scores
print("R-squared score for training data:", r2_train)
print("R-squared score for testing data:", r2_test)

R-squared score for training data: 0.9769288175754218
R-squared score for testing data: 0.9041984175586777


### 11. Create a pickle file with an extension as .pkl

In [24]:
import pickle
# Saving model to disk
pickle.dump(rfr, open('model.pkl','wb'))

### 12. Create new folder/new project in visual studio/pycharm that should contain the "model.pkl" file *make sure you are using a virutal environment and install required packages.*

### a) Create a basic HTML form for the frontend

Create a file **index.html** in the templates folder and copy the following code.

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8" />
    <title>ML Deployment</title>
    <link
      href="https://fonts.googleapis.com/css?family=Pacifico"
      rel="stylesheet"
      type="text/css"
    />
    <link
      href="https://fonts.googleapis.com/css?family=Arimo"
      rel="stylesheet"
      type="text/css"
    />
    <link
      href="https://fonts.googleapis.com/css?family=Hind:300"
      rel="stylesheet"
      type="text/css"
    />
    <link
      href="https://fonts.googleapis.com/css?family=Open+Sans+Condensed:300"
      rel="stylesheet"
      type="text/css"
    />
    <link
      rel="stylesheet"
      href="{{ url_for('static', filename='style.css') }}"
    />
  </head>
  <body>
    <div class="login">
      <h1>Car Price Predictor</h1>
      <form action="{{ url_for('predict')}}" method="post">
        <h2>Enter Car Details:</h2>
        <br />

        <h3>Age of car in (In Years)</h3>
        <input
          id="first"
          type="number"
          name="Age_of_the_car"
          placeholder="Enter Age of car"
          required="required"
        />
        <br />
        <h3>Present showroom Price (In lakhs)</h3>

        <input
          id="second"
          name="Present_Price"
          required="required"
          placeholder="Enter showroom price"
        />
        <br />
        <h3>Kilometers Driven</h3>
        <input
          id="third"
          name="Kms_Driven"
          required="required"
          placeholder="Enter Kms Driven"
        />

        <h3>Owner Type (0/1/3)</h3>

        <input
          id="fourth"
          name="Owner"
          required="required"
          placeholder="Enter Owner Type"
        />
        <br />
        <h3>Fuel Type</h3>

        <select name="Fuel_Type" id="fuel" required="required">
          <option value="0">Petrol</option>
          <option value="1">Diesel</option>
          <option value="2">CNG</option>
        </select>
        <br />
        <h3>Seller Type</h3>

        <select name="Seller_Type" id="resea" required="required">
          <option value="0">Dealer</option>
          <option value="1">Individual</option>
        </select>
        <br />
        <h3>Transmission Type</h3>

        <select name="Transmission" id="research" required="required">
          <option value="0">Manual Car</option>
          <option value="1">Automatic Car</option>
        </select>

        <br />
        <br />
        <br />
        <button type="submit" class="btn btn-primary btn-block btn-large">
          Predict
        </button>
      </form>
      <br />
      <br />
      <div class="pred-text">{{ prediction_text }}</div>
    </div>
  </body>
</html>


### b) Create app.py file and write the predict function

In [ ]:
# importing necessary libraries and functions
import numpy as np
from flask import Flask, request, jsonify, render_template
import pickle
import sklearn

#Initializing the flask App
app = Flask(__name__) 
# loading the trained model
model = pickle.load(open('model.pkl', 'rb')) 

# Homepage
@app.route('/', methods=['GET']) 
def Home():
    return render_template('index.html')

# Predicting car price
@app.route('/predict',methods=['POST'])
def predict():
    if request.method == 'POST':
        Present_Price=float(request.form['Present_Price'])
        Kms_Driven=int(request.form['Kms_Driven'])
        Owner=int(request.form['Owner'])
        Fuel_Type=request.form['Fuel_Type']
        Age_of_the_car=request.form['Age_of_the_car']
        Seller_Type=request.form['Seller_Type']
        Transmission=request.form['Transmission']

        prediction=model.predict([[Present_Price, Kms_Driven, Owner, Age_of_the_car, Fuel_Type, Seller_Type, Transmission]])
        output= round(prediction[0],2)
        return render_template('index.html', prediction_text="You can sell your car at {} lakhs". format(output))

if __name__ == "__main__":
    app.run(debug=True)

### - optional CSS file (style.css) in static folder



In [ ]:
@import url(https://fonts.googleapis.com/css?family=Open+Sans);
.btn {
  display: inline-block;
  padding: 4px 10px 4px;
  margin-bottom: 0;
  font-size: 13px;
  line-height: 18px;
  color: #333333;
  text-align: center;
  text-shadow: 0 1px 1px rgba(255, 255, 255, 0.75);
  vertical-align: middle;
  background-color: #f5f5f5;
  background-image: -moz-linear-gradient(top, #ffffff, #e6e6e6);
  background-image: -ms-linear-gradient(top, #ffffff, #e6e6e6);
  background-image: -webkit-gradient(
    linear,
    0 0,
    0 100%,
    from(#ffffff),
    to(#e6e6e6)
  );
  background-image: -webkit-linear-gradient(top, #ffffff, #e6e6e6);
  background-image: -o-linear-gradient(top, #ffffff, #e6e6e6);
  background-image: linear-gradient(top, #ffffff, #e6e6e6);
  background-repeat: repeat-x;
  filter: progid:dximagetransform.microsoft.gradient(startColorstr=#ffffff, endColorstr=#e6e6e6, GradientType=0);
  border-color: #e6e6e6 #e6e6e6 #e6e6e6;
  border-color: rgba(0, 0, 0, 0.1) rgba(0, 0, 0, 0.1) rgba(0, 0, 0, 0.25);
  border: 1px solid #e6e6e6;
  -webkit-border-radius: 4px;
  -moz-border-radius: 4px;
  border-radius: 4px;
  -webkit-box-shadow: inset 0 1px 0 rgba(255, 255, 255, 0.2),
    0 1px 2px rgba(0, 0, 0, 0.05);
  -moz-box-shadow: inset 0 1px 0 rgba(255, 255, 255, 0.2),
    0 1px 2px rgba(0, 0, 0, 0.05);
  box-shadow: inset 0 1px 0 rgba(255, 255, 255, 0.2),
    0 1px 2px rgba(0, 0, 0, 0.05);
  cursor: pointer;
  margin-left: 0.3em;
}
.btn:hover,
.btn:active,
.btn.active,
.btn.disabled,
.btn[disabled] {
  background-color: #e6e6e6;
}
.btn-large {
  padding: 9px 14px;
  font-size: 15px;
  line-height: normal;
  -webkit-border-radius: 5px;
  -moz-border-radius: 5px;
  border-radius: 5px;
}
.btn:hover {
  color: #333333;
  text-decoration: none;
  background-color: #e6e6e6;
  background-position: 0 -15px;
  -webkit-transition: background-position 0.1s linear;
  -moz-transition: background-position 0.1s linear;
  -ms-transition: background-position 0.1s linear;
  -o-transition: background-position 0.1s linear;
  transition: background-position 0.1s linear;
}
.btn-primary,
.btn-primary:hover {
  text-shadow: 0 -1px 0 rgba(0, 0, 0, 0.25);
  color: #ffffff;
}
.btn-primary.active {
  color: rgba(255, 255, 255, 0.75);
}
.btn-primary {
  background-color: #4a77d4;
  background-image: -moz-linear-gradient(top, #6eb6de, #4a77d4);
  background-image: -ms-linear-gradient(top, #6eb6de, #4a77d4);
  background-image: -webkit-gradient(
    linear,
    0 0,
    0 100%,
    from(#6eb6de),
    to(#4a77d4)
  );
  background-image: -webkit-linear-gradient(top, #6eb6de, #4a77d4);
  background-image: -o-linear-gradient(top, #6eb6de, #4a77d4);
  background-image: linear-gradient(top, #6eb6de, #4a77d4);
  background-repeat: repeat-x;
  filter: progid:dximagetransform.microsoft.gradient(startColorstr=#6eb6de, endColorstr=#4a77d4, GradientType=0);
  border: 1px solid #3762bc;
  text-shadow: 1px 1px 1px rgba(0, 0, 0, 0.4);
  box-shadow: inset 0 1px 0 rgba(255, 255, 255, 0.2),
    0 1px 2px rgba(0, 0, 0, 0.5);
}

* {
  -webkit-box-sizing: border-box;
  -moz-box-sizing: border-box;
  -ms-box-sizing: border-box;
  -o-box-sizing: border-box;
  box-sizing: border-box;
}

html {
  width: 100%;
  height: 100%;
}

body {
  width: 100%;
  height: 100%;
  font-family: "Open Sans", sans-serif;
  background: #092756;
  color: #fff;
  font-size: 18px;
  text-align: center;
  letter-spacing: 1.2px;
}

.login {
  position: absolute;
  top: 30%;
  left: 50%;
  margin: -150px 0 0 -150px;
  width: 400px;
  height: 400px;
}

.login h1 {
  color: #fff;
  text-shadow: 0 0 10px rgba(0, 0, 0, 0.3);
  letter-spacing: 1px;
  text-align: center;
}

input {
  width: 100%;
  margin-bottom: 20px;
  background: rgba(0, 0, 0, 0.3);
  border: none;
  outline: none;
  padding: 10px;
  font-size: 13px;
  color: #fff;
  text-shadow: 1px 1px 1px rgba(0, 0, 0, 0.3);
  border: 1px solid rgba(0, 0, 0, 0.3);
  border-radius: 4px;
  box-shadow: inset 0 -5px 45px rgba(100, 100, 100, 0.2),
    0 1px 1px rgba(255, 255, 255, 0.2);
  -webkit-transition: box-shadow 0.5s ease;
  -moz-transition: box-shadow 0.5s ease;
  -o-transition: box-shadow 0.5s ease;
  -ms-transition: box-shadow 0.5s ease;
  transition: box-shadow 0.5s ease;
}

input:focus {
  box-shadow: inset 0 -5px 45px rgba(100, 100, 100, 0.4),
    0 1px 1px rgba(255, 255, 255, 0.2);
}

select {
  width: 100%;
  margin-bottom: 20px;
  background: rgba(0, 0, 0, 0.3);
  border: none;
  outline: none;
  padding: 10px;
  font-size: 13px;
  color: #fff;
  text-shadow: 1px 1px 1px rgba(0, 0, 0, 0.3);
  border: 1px solid rgba(0, 0, 0, 0.3);
  border-radius: 4px;
  box-shadow: inset 0 -5px 45px rgba(100, 100, 100, 0.2),
    0 1px 1px rgba(255, 255, 255, 0.2);
  -webkit-transition: box-shadow 0.5s ease;
  -moz-transition: box-shadow 0.5s ease;
  -o-transition: box-shadow 0.5s ease;
  -ms-transition: box-shadow 0.5s ease;
  transition: box-shadow 0.5s ease;
}

.pred-text {
  font-size: 30px;
  color: white;
  margin-bottom: 50px;
  padding-bottom: 30px;
}


### 13. Run the app.py python file which will render to index html page then enter the input values and get the prediction.

- The app.py file is run on following link: 
http://127.0.0.1:5000

**Predition Case 1:** <br>
Age of car in Years - 2 <br>
Present showroom price - 20 <br>
Kilometers Driven - 1000 <br>
Owner Type (0/1/3) - 0 <br>
Fuel Type - Petrol <br>
Seller Type - Individual <br>
Transmission Type - Automatic Car <br>
 
**Predition Result =  You can sell your car at 16.45 lakhs**

**Predition Case 2:** <br>
Age of car in Years - 10 <br>
Present showroom price - 10 <br>
Kilometers Driven - 5000 <br>
Owner Type (0/1/3) - 3 <br>
Fuel Type - Diesel <br>
Seller Type - Dealer <br>
Transmission Type - Manual Car <br>
 
**Predition Result =  You can sell your car at 7.86 lakhs**

### Happy Learning :)